In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2
import time

In [7]:
VIDEO_PATH = "D:/editor/DFD/process"  # Parent folder containing "real/" and "fake/"
FRAME_SIZE = (224, 224)  # Resizing frames
NUM_FRAMES = 30  # Number of frames to sample per video
BATCH_SIZE = 32
EPOCHS = 20

In [8]:
def extract_frames(video_path, num_frames=NUM_FRAMES):
    """Extract a fixed number of frames from a video."""
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    for idx in range(total_frames):
        ret, frame = cap.read()
        if idx in frame_indices and ret:
            frame = cv2.resize(frame, FRAME_SIZE)
            frames.append(frame)
    cap.release()
    return np.array(frames)

def load_data_from_folders(video_folder):
    """Load videos and their labels from separate folders."""
    data, labels = [], []
    for label, subfolder in enumerate(['real', 'fake']):  # 0 = real, 1 = fake
        folder_path = os.path.join(video_folder, subfolder)
        for video_name in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video_name)
            frames = extract_frames(video_path)
            if len(frames) == NUM_FRAMES:
                data.append(frames)
                labels.append(label)
    return np.array(data), np.array(labels)

In [9]:
print("Loading data...")
start_time = time.time()
data, labels = load_data_from_folders(VIDEO_PATH)
labels = np.array(labels, dtype=np.float32)
data = data / 255.0  # Normalize pixel values to [0, 1]
print(f"Data shape: {data.shape}, Labels shape: {labels.shape}")

Loading data...


MemoryError: Unable to allocate 14.4 GiB for an array with shape (3419, 30, 224, 224, 3) and data type uint8

In [ ]:
cnn_base = ResNet50(include_top=False, input_shape=(224, 224, 3), pooling='avg')
cnn_base.trainable = False  # Freeze the CNN

def extract_cnn_features(frames):
    """Extract features for a sequence of frames."""
    features = np.array([cnn_base.predict(frame[None, ...])[0] for frame in frames])  # Use [0] to remove the extra axis
    return features

In [ ]:
print("Extracting CNN features...")
cnn_features = np.array([extract_cnn_features(video) for video in data])
print(f"Feature shape: {cnn_features.shape}")  # Ensure this is (num_videos, NUM_FRAMES, 2048)

In [ ]:
num_features = cnn_features.shape[-1]

model = models.Sequential([
    layers.Input(shape=(NUM_FRAMES, num_features)),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
print("Training model...")
history = model.fit(
    cnn_features, labels,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

In [ ]:
print("Evaluating model...")
loss, accuracy = model.evaluate(cnn_features, labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
end_time = time.time()
print(f'Total time for training {(end_time-start_time):.3f} seconds')

In [ ]:
import matplotlib.pyplot as plt

# Extract loss and accuracy from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot Loss
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend()
plt.show()

# Plot Accuracy
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend()
plt.show()

In [ ]:
model.save("deepfake_detection_model.h5")
print("Model saved successfully!")

In [ ]:
model.save("deepfake_detection_model.keras")
print("Model saved successfully!")

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

FRAME_SIZE = (224, 224)  # Resizing frames
NUM_FRAMES = 30  # Number of frames to sample per video
MODEL_PATH = "deepfake_detection_model.h5"  # Path to the trained model

def extract_frames(video_path, num_frames=NUM_FRAMES):
    """Extract a fixed number of frames from a test video."""
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
    for idx in range(total_frames):
        ret, frame = cap.read()
        if idx in frame_indices and ret:
            frame = cv2.resize(frame, FRAME_SIZE)
            frames.append(frame)
    cap.release()
    return np.array(frames) / 255.0  # Normalize pixel values to [0, 1]

model = load_model(MODEL_PATH)

test_video_path = "D:/editor/DFD/Fake/01_03__hugging_happy__ISF9SP4G.mp4"  # Replace with the actual test video path
print(f"Testing video: {test_video_path}")

frames = extract_frames(test_video_path)

if len(frames) == NUM_FRAMES:
    # Extract features using the CNN base
    from tensorflow.keras.applications import ResNet50
    cnn_base = ResNet50(include_top=False, input_shape=(224, 224, 3), pooling='avg')
    cnn_base.trainable = False  # Ensure the CNN is not trainable during testing

    features = np.array([cnn_base.predict(frame[None, ...])[0] for frame in frames])
    features = np.expand_dims(features, axis=0)  # Add batch dimension for model input

    # Predict the class of the test video
    prediction = model.predict(features)
    print(prediction)
    print("Prediction:", "Real" if prediction[0] < 0.5 else "Fake")
else:
    print(f"Insufficient frames extracted. Expected {NUM_FRAMES}, got {len(frames)}.")